### Entanglement-based Quantum Key Distribution in a $n$ x $n$ grid topology using Netsquid

#### The objetive of this project is to use the Ekert Protocol in order to implement QKD in a $n$ x $n$ Quantum Network. The following list describes the milestones to be achieved:
* Entangle a random route using Entanglement Swapping (Completed)
* Entangle two or more disjunctive routes (Completed)
* Generate the shared key in each node (In process)  

#### Code functionality: This implementation creates a $n$ x $n$ grid Quantum Network. After the Quantum Network is defined, the simulation starts dynamic protocols depending on the connection requirements. When the simulation ends, it brings back the path, fidelity and time information.

_refs_: 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html 
* Evan Sutcliffe, Matty J. Hoban & Alejandra Beghelli - Multipath Routing for Multipartite State Distribution in Quantum Networks  

_(adapted by D-Cryp7 for Netsquid 1.1.6)_

#### Limitations:
* Conjunctive routes causes a Race Condition error. For now, our aim focus on disjunctive routes, due to the possible alternative options.

In [1]:
# Library imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
sys.path.insert(0, '../')
from custom_netsquid_functions import *
from base_netsquid_functions import *
from util import *

In [2]:
def setup_datacollector(network, protocol, path, node_qconnections):
    """
    Setup the datacollector to calculate the fidelity
    when the CorrectionProtocol has finished.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    protocol : :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Datacollector recording fidelity data.

    """

    # Ensure nodes are ordered in the chain:
    # nodes = [network.nodes[name] for name in sorted(network.nodes.keys())]
    nodes = [network.nodes[str(name)] for name in path]
    A_port = int(node_qconnections[nodes[0].name][nodes[1].name].split("-")[0][-1])
    B_port = int(node_qconnections[nodes[-1].name][nodes[-2].name].split("-")[0][-1])
    nodes[0].shared_secret, nodes[-1].shared_secret = [], []
    
    nodes[0].entangled_qubits, nodes[-1].entangled_qubits = {}, {}
    entangled = False
    entangle_attemps = 0
    def calc_fidelity(evexpr):
        nonlocal entangled, entangle_attemps
        qubit_a, = nodes[0].qmemory.peek([A_port])
        qubit_b, = nodes[-1].qmemory.peek([B_port])
        fidelity = ns.qubits.fidelity([qubit_a, qubit_b], ks.b00, squared = True)
        measure_a = measure(qubit_a, random_basis())
        measure_b = measure(qubit_b, random_basis())
        if fidelity >= 0.9:
            if not entangled:
                # print(f"{path} entangled in {entangle_attemps} attemps: storing qubits")
                nodes[0].entangled_qubits[f"{nodes[-1].name}"] = qubit_a
                nodes[-1].entangled_qubits[f"{nodes[0].name}"] = qubit_b
                entangled = True
            # QKD part
            if measure_a[0] == measure_b[0]:
                nodes[0].shared_secret.append(str(int(measure_a[1])))
                nodes[-1].shared_secret.append(str(int(measure_b[1])))
        else:
            entangle_attemps += 1
        return {"fidelity": fidelity, 
                "A_measure": measure_a[0], "A_measure_result": measure_a[1], 
                "B_measure": measure_b[0], "B_measure_result": measure_b[1]}

    dc = DataCollector(calc_fidelity, include_entity_name=False)
    dc.collect_on(pydynaa.EventExpression(source=protocol.subprotocols['CorrectProtocol'],
                                          event_type=Signals.SUCCESS.value))
    return dc

#### Application: Generate a shared secret between two nodes (QKD)

In [55]:
%%time

import time

# Default values

n = 10
node_distance = 20
num_iters = 256
num_node = 10
est_runtime = node_distance * 5e3


ns.sim_reset()
network, node_qconnections = network_setup(n, node_distance = node_distance,
                        source_frequency = 1e9 / est_runtime)

qdf = pandas.DataFrame(node_qconnections)
qdf = qdf.fillna("")

path = get_random_route(network)

while len(path) != num_node:
    path = get_random_route(network)

traffic = {
    "path": [path]
}

print(f"Simulation from path: {path}")

df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)

assert network.nodes[str(path[0])].shared_secret == network.nodes[str(path[-1])].shared_secret

print(f'Shared key bitsring: {"".join(network.nodes[str(path[0])].shared_secret)}')

network.nodes[str(path[0])].shared_secret = sha256(long_to_bytes(int(''.join(network.nodes[str(path[0])].shared_secret), 2))).digest()
network.nodes[str(path[-1])].shared_secret = sha256(long_to_bytes(int(''.join(network.nodes[str(path[-1])].shared_secret), 2))).digest()

print(f"Shared key SHA256: {network.nodes[str(path[0])].shared_secret.hex()}")

m = b"Qu4nTuM_k3Y_D1s7r1buT10n!"
cipher = AES.new(network.nodes[str(path[0])].shared_secret, AES.MODE_ECB)
encrypted_data = cipher.encrypt(pad(m, 16))
print(f"Alice encrypted data: {encrypted_data.hex()}")

cipher = AES.new(network.nodes[str(path[-1])].shared_secret, AES.MODE_ECB)
decrypted_data = unpad(cipher.decrypt(encrypted_data), 16)
print(f"Bob decrypted data: {decrypted_data.decode()}")

Simulation from path: [(2, 2), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8)]
Shared key bitsring: 11000
Shared key SHA256: 452ba1ddef80246c48be7690193c76c1d61185906be9401014fe14f1be64b74f
Alice encrypted data: 56dd312e83dc30d58ae777db9205285b8ea7030a8809c3472cf07ee62a78a284
Bob decrypted data: Qu4nTuM_k3Y_D1s7r1buT10n!
CPU times: user 25.7 s, sys: 44.2 ms, total: 25.7 s
Wall time: 25.7 s


In [49]:
df[0].dataframe.head(5)

,time_stamp,fidelity,A_measure,A_measure_result,B_measure,B_measure_result
0,150001.0,1.0,X,0.0,Z,0.0
1,250001.0,0.0,X,0.0,X,1.0
2,350002.0,1.0,X,1.0,X,1.0
3,450003.0,1.0,X,0.0,X,0.0
4,550001.0,0.0,Z,0.0,X,1.0


In [ ]:
"""Run the simulation for multiple nodes and distances and show them in a figure.

Parameters
----------

num_iters : int, optional
    Number of iterations per simulation configuration.
    At least 1. Default 2000.
"""
from matplotlib import pyplot as plt
n = 10
num_iters = 1024

result = pandas.DataFrame([], index = range(2, 20), columns = [10, 30, 50, 70, 100, 150, 200])

fig, ax = plt.subplots()
for distance in [10, 30, 50, 70, 100, 150, 200]:
    data = pandas.DataFrame()
    for num_node in range(2, 20):
        ns.sim_reset()
        est_runtime = distance * 5e3
        network, node_qconnections = network_setup(n, node_distance = distance,
                                source_frequency = 1e9 / est_runtime)
        qdf = pandas.DataFrame(node_qconnections)
        qdf = qdf.fillna("")
        path = get_random_route(network)
        while len(path) != num_node:
            path = get_random_route(network)
        traffic = {
            "path": [path]
        }
        print(f"Simulation from path: {path}")
        data[num_node] = run_simulation(network, qdf,
                                        est_runtime, num_iters,
                                        traffic, setup_datacollector)[0].dataframe['fidelity']
        # print(len(data[num_node]))
        result[distance][num_node] = list(data[num_node])
        result.to_csv("results.csv")
    # For errorbars we use the standard error of the mean (sem)
    # data.fillna(0.0)
    # data = data.agg(['mean', 'sem']).T.rename(columns={'mean': 'fidelity'})
    # data.plot(y='fidelity', yerr='sem', label=f"{distance} km", ax=ax)
# plt.xlabel("path length (number of nodes)")
# plt.ylabel("fidelity")
# plt.title("Repeater chain with different total lengths")
# plt.show()

In [ ]:
"""Run the simulation for multiple nodes and distances and show them in a figure.

Parameters
----------

num_iters : int, optional
    Number of iterations per simulation configuration.
    At least 1. Default 2000.
"""
from matplotlib import pyplot as plt
import time
n = 10
num_iters = 1024

more_results = pandas.DataFrame([], index = range(2, 20), columns = [10, 30, 50, 70, 100, 150, 200])
exec_time_results = pandas.DataFrame([], index = range(2, 20), columns = [10, 30, 50, 70, 100, 150, 200])
# network_time_results = pandas.DataFrame([], index = range(2, 20), columns = [10, 30, 50, 70, 100, 150, 200])

for distance in [10, 30, 50, 70, 100, 150, 200]:
    data = pandas.DataFrame()
    for num_node in range(2, 20):
        ns.sim_reset()
        est_runtime = distance * 5e3
        network, node_qconnections = network_setup(n, node_distance = distance,
                                source_frequency = 1e9 / est_runtime)
        qdf = pandas.DataFrame(node_qconnections)
        qdf = qdf.fillna("")
        path = get_random_route(network)
        while len(path) != num_node:
            path = get_random_route(network)
        traffic = {
            "path": [path]
        }
        print(f"Simulation from path: {path}")
        start_time = time.time()
        data[num_node] = run_simulation(network, qdf,
                                        est_runtime, num_iters,
                                        traffic, setup_datacollector)[0].dataframe['fidelity']
        end_time = time.time() - start_time
        exec_time_results[distance][num_node] = end_time
        print(f"Simulation time: {end_time}")
        # print(len(data[num_node]))
        more_results[distance][num_node] = list(data[num_node])
        more_results.to_csv("more_results.csv")
        exec_time_results.to_csv("time_results.csv")

In [4]:
exec_time_results = pandas.read_csv("execution_time_results.csv", index_col = 0)
# exec_time_results["3"][12] = 0.0
# exec_time_results.to_csv("execution_time_results.csv")

In [4]:
n = 4

network, node_qconnections = network_setup(n, node_distance = distance,
                        source_frequency = 1e9 / est_runtime)
qdf = pandas.DataFrame(node_qconnections)
qdf = qdf.fillna("")
path = get_random_route(network)
while len(path) != 10:
    path = get_random_route(network)
    print(path)

KeyboardInterrupt: 

In [3]:
import time

num_iters = 256
distance = 20
# num_node = 3

# exec_time_results = pandas.DataFrame([], index = range(3, 101), columns = range(3, 101))
# exec_time_results = exec_time_results.fillna(0.0)
# for n in range(3, 101)


for num_node in range(14, 21):
    for n in range(7, 11):
        for i in range(100):
            ns.sim_reset()
            est_runtime = distance * 5e3
            network, node_qconnections = network_setup(n, node_distance = distance,
                                    source_frequency = 1e9 / est_runtime)
            qdf = pandas.DataFrame(node_qconnections)
            qdf = qdf.fillna("")
            path = get_random_route(network)
            while len(path) != num_node:
                path = get_random_route(network)
            traffic = {
                "path": [path]
            }
            # print(f"Simulation from path: {path}")
            start_time = time.time()
            df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
            end_time = time.time() - start_time
            exec_time_results[str(num_node)][n] += end_time / 100
            exec_time_results.to_csv("execution_time_results.csv")
            print(f"{i}-Simulation time for a {num_node}-node length in a {n}x{n} GQN: {end_time}")
        print(f"Mean simulation time for a {num_node}-node length in a {n}x{n} GQN: {exec_time_results[str(num_node)][n]}")

KeyboardInterrupt: 

In [3]:
common_paths = {
    "3": [(0, 0), (1, 0), (1, 1)],
    "4": [(0, 0), (1, 0), (1, 1), (0, 1)],
    "5": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2)],
    "6": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2)],
    "7": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2)],
    "8": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1)],
    "9": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0)],
    "10": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0)],
    "11": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1)],
    "12": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2)],
    "13": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3)],
    "14": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3)],
    "15": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3)],
    "16": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)],
    "17": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4)],
    "18": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4), (1, 4)],
    "19": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4), (1, 4), (2, 4)],
    "20": [(0, 0), (1, 0), (1, 1), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4)]
}

In [14]:
import time

num_iters = 256
distance = 20
# num_node = 3

# exec_time_results = pandas.DataFrame([], index = range(3, 101), columns = range(3, 101))
# exec_time_results = exec_time_results.fillna(0.0)
# for n in range(3, 101)

exec_time_results = pandas.read_csv("execution_time_results.csv", index_col = 0)

for num_node in [18]:
    for n in range(5, 11):
        ns.sim_reset()
        est_runtime = distance * 5e3
        network, node_qconnections = network_setup(n, node_distance = distance,
                                source_frequency = 1e9 / est_runtime)
        qdf = pandas.DataFrame(node_qconnections)
        qdf = qdf.fillna("")
        '''
        path = get_random_route(network)
        while len(path) != num_node:
            path = get_random_route(network)
        traffic = {
            "path": [path]
        }
        '''
        traffic = {
            "path": [common_paths[str(num_node)]]
        }
        # print(f"Simulation from path: {path}")
        start_time = time.time()
        df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
        end_time = time.time() - start_time
        exec_time_results[str(num_node)][n] += end_time
        exec_time_results.to_csv("execution_time_results.csv")
        print(f"Simulation time for a {num_node}-node length in a {n}x{n} GQN: {end_time}")

Simulation time for a 18-node length in a 5x5 GQN: 6.02576208114624
Simulation time for a 18-node length in a 6x6 GQN: 8.592207431793213
Simulation time for a 18-node length in a 7x7 GQN: 11.646414995193481
Simulation time for a 18-node length in a 8x8 GQN: 14.730713605880737
Simulation time for a 18-node length in a 9x9 GQN: 24.446991205215454
Simulation time for a 18-node length in a 10x10 GQN: 26.460809469223022


In [5]:
import time

num_iters = 256
distance = 20
# num_node = 3

# exec_time_results = pandas.DataFrame([], index = range(3, 101), columns = range(3, 101))
# exec_time_results = exec_time_results.fillna(0.0)
# for n in range(3, 101)


for num_node in [18]:
    for n in range(16, 19):
        for i in range(10):
            ns.sim_reset()
            est_runtime = distance * 5e3
            network, node_qconnections = network_setup(n, node_distance = distance,
                                    source_frequency = 1e9 / est_runtime)
            qdf = pandas.DataFrame(node_qconnections)
            qdf = qdf.fillna("")
            '''
            path = get_random_route(network)
            while len(path) != num_node:
                path = get_random_route(network)
            traffic = {
                "path": [path]
            }
            '''
            traffic = {
                "path": [common_paths[str(num_node)]]
            }
            # print(f"Simulation from path: {path}")
            start_time = time.time()
            df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
            end_time = time.time() - start_time
            exec_time_results[str(num_node)][n] += end_time / 10
            exec_time_results.to_csv("execution_time_results.csv")
            print(f"{i}-Simulation time for a {num_node}-node length in a {n}x{n} GQN: {end_time}")
        print(f"Mean simulation time for a {num_node}-node length in a {n}x{n} GQN: {exec_time_results[str(num_node)][n]}")

0-Simulation time for a 8-node length in a 15x15 GQN: 59.1712486743927
1-Simulation time for a 8-node length in a 15x15 GQN: 61.13888883590698
2-Simulation time for a 8-node length in a 15x15 GQN: 60.123045444488525
3-Simulation time for a 8-node length in a 15x15 GQN: 60.5719780921936
4-Simulation time for a 8-node length in a 15x15 GQN: 60.183000326156616
5-Simulation time for a 8-node length in a 15x15 GQN: 59.422632694244385
6-Simulation time for a 8-node length in a 15x15 GQN: 59.415358781814575
7-Simulation time for a 8-node length in a 15x15 GQN: 60.1324257850647
8-Simulation time for a 8-node length in a 15x15 GQN: 59.65064287185669
9-Simulation time for a 8-node length in a 15x15 GQN: 60.455289363861084
Mean simulation time for a 8-node length in a 15x15 GQN: 60.02645108699798
0-Simulation time for a 8-node length in a 16x16 GQN: 67.42862844467163
1-Simulation time for a 8-node length in a 16x16 GQN: 65.46165537834167
2-Simulation time for a 8-node length in a 16x16 GQN: 66.66

In [4]:
import time

num_iters = 256
distance = 20
num_node = 8

exec_time_results = pandas.read_csv("execution_time_results.csv", index_col = 0)

# exec_time_results = pandas.DataFrame([], index = range(3, 101), columns = range(3, 101))
# exec_time_results = exec_time_results.fillna(0.0)
# for n in range(3, 101)
# 17 - 10, 100
# 18 - 90, 100

n = 14

traffic = {
    "path": [common_paths[str(num_node)]]
}

for i in range(2, 10):
    ns.sim_reset()
    est_runtime = distance * 5e3
    network, node_qconnections = network_setup(n, node_distance = distance,
                            source_frequency = 1e9 / est_runtime)
    qdf = pandas.DataFrame(node_qconnections)
    qdf = qdf.fillna("")
    '''
    path = get_random_route(network)
    while len(path) != num_node:
        path = get_random_route(network)
    traffic = {
        "path": [path]
    }
    '''
    # print(f"Simulation from path: {path}")
    start_time = time.time()
    df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
    end_time = time.time() - start_time
    exec_time_results[str(num_node)][n] += end_time / 10
    exec_time_results.to_csv("execution_time_results.csv")
    print(f"{i}-Simulation time for a {num_node}-node length in a {n}x{n} GQN: {end_time}")
print(f"Mean simulation time for a {num_node}-node length in a {n}x{n} GQN: {exec_time_results[str(num_node)][n]}")

2-Simulation time for a 8-node length in a 14x14 GQN: 55.005370140075684
3-Simulation time for a 8-node length in a 14x14 GQN: 55.41710114479065
4-Simulation time for a 8-node length in a 14x14 GQN: 55.146321296691895
5-Simulation time for a 8-node length in a 14x14 GQN: 56.54296040534973
6-Simulation time for a 8-node length in a 14x14 GQN: 55.92134380340576
7-Simulation time for a 8-node length in a 14x14 GQN: 51.57936644554138
8-Simulation time for a 8-node length in a 14x14 GQN: 50.97239351272583
9-Simulation time for a 8-node length in a 14x14 GQN: 51.09452962875366
Mean simulation time for a 8-node length in a 14x14 GQN: 55.50686736106873


XD

In [3]:
import time

num_iters = 256
distance = 20
num_node = 14

exec_time_results = pandas.read_csv("execution_time_results.csv", index_col = 0)

# exec_time_results = pandas.DataFrame([], index = range(3, 101), columns = range(3, 101))
# exec_time_results = exec_time_results.fillna(0.0)
# for n in range(3, 101)
# 17 - 10, 100
# 18 - 90, 100

n = 10

for i in range(80, 101):
    ns.sim_reset()
    est_runtime = distance * 5e3
    network, node_qconnections = network_setup(n, node_distance = distance,
                            source_frequency = 1e9 / est_runtime)
    qdf = pandas.DataFrame(node_qconnections)
    qdf = qdf.fillna("")
    path = get_random_route(network)
    while len(path) != num_node:
        path = get_random_route(network)
    traffic = {
        "path": [path]
    }
    # print(f"Simulation from path: {path}")
    start_time = time.time()
    df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
    end_time = time.time() - start_time
    exec_time_results[str(num_node)][n] += end_time / 100
    exec_time_results.to_csv("execution_time_results.csv")
    print(f"{i}-Simulation time for a {num_node}-node length in a {n}x{n} GQN: {end_time}")
print(f"Mean simulation time for a {num_node}-node length in a {n}x{n} GQN: {exec_time_results[str(num_node)][n]}")

80-Simulation time for a 13-node length in a 10x10 GQN: 27.856959581375122
81-Simulation time for a 13-node length in a 10x10 GQN: 26.51586389541626
82-Simulation time for a 13-node length in a 10x10 GQN: 26.83255362510681
83-Simulation time for a 13-node length in a 10x10 GQN: 26.362199306488037
84-Simulation time for a 13-node length in a 10x10 GQN: 28.260751724243164
85-Simulation time for a 13-node length in a 10x10 GQN: 27.253610134124756
86-Simulation time for a 13-node length in a 10x10 GQN: 27.579110860824585
87-Simulation time for a 13-node length in a 10x10 GQN: 27.15430450439453
88-Simulation time for a 13-node length in a 10x10 GQN: 27.788200855255127
89-Simulation time for a 13-node length in a 10x10 GQN: 28.133859634399414
90-Simulation time for a 13-node length in a 10x10 GQN: 26.920668601989746
91-Simulation time for a 13-node length in a 10x10 GQN: 27.366405487060547
92-Simulation time for a 13-node length in a 10x10 GQN: 26.32501792907715
93-Simulation time for a 13-n

In [ ]:
import time

n = 5
num_iters = 1024
distance = 20

num_node = 3

ns.sim_reset()
est_runtime = distance * 5e3
network, node_qconnections = network_setup(n, node_distance = distance,
                        source_frequency = 1e9 / est_runtime)
qdf = pandas.DataFrame(node_qconnections)
qdf = qdf.fillna("")
path = get_random_route(network)
while len(path) != num_node:
    path = get_random_route(network)
path = [(0, 0), (1, 0), (2, 0)]
traffic = {
    "path": [path]
}
print(f"Simulation from path: {path}")

start_time = time.time()
df = run_simulation(network, qdf, est_runtime, num_iters, traffic, setup_datacollector)
end_time = time.time() - start_time
print(f"Simulation time: {end_time}")

In [ ]:
df[0].dataframe

##### Work In Progress (WIP): Analyse the secret key rate (SKR)

In [ ]:
dataframe = df[0].dataframe
fid_df = dataframe[dataframe["fidelity"] >= 0.9]
Z_fid = fid_df[fid_df["B_measure"] == "Z"]
X_fid = fid_df[fid_df["B_measure"] == "X"]
Q_z = Z_fid[Z_fid["A_measure"] != Z_fid["B_measure"]].shape[0] / Z_fid.shape[0]
Q_x = X_fid[X_fid["A_measure"] != X_fid["B_measure"]].shape[0] / X_fid.shape[0]
print(f"QBER in Z basis: {Q_z}")
print(f"QBER in X basis: {Q_x}")
1 - binary_entropy(Q_x) - binary_entropy(Q_z) # some part of the SKR